# Instructions
- If you would like to modify this notebook, please make a copy and make your modifications there
- Make sure to backup your notebook on your computer or on github


# Assumptions
- All output from postprocessor is in one json file (output.json) 
- All in scope new sources are in one textfile (scope.txt)
    - one news source per line
- If any of these change then you would have to modify how the input is read

In [31]:
import ijson
import os
import pandas as pd
import functools
from datetime import datetime

specify scope and json file

In [32]:
SCOPE ='./scope.txt'
JSON_WITH_IDS='../../json_data/output.json'

In [33]:
print(os.getcwd())

/media/data/jupyter-lab/bin/jupyter_notebooks/csv_processing


open source text file and store them in the `scope` array

In [34]:
# get scope from textfile
scope = []
f= open(SCOPE,"r")
lines = f.readlines()
for line in lines:
    scope.append(line.replace('\n',''))

In [35]:
scope

['972mag',
 'Arutz 7 - English',
 'Arutz 7 - Hebrew',
 'Calcalist',
 'Debka File - English',
 'Debka File - Hebrew',
 'GAZA NEWS 48',
 'Globes',
 'Globes - English',
 'Haaretz - English',
 'Haaretz - Hebrew',
 'HaModia',
 'Israel Hayom - English',
 'Israel Hayom - Hebrew',
 'Israeli Government Press Office - English',
 'Israeli Government Press Office - Hebrew',
 'Jerusalem Post',
 'Jewish Ultra-Orthodox News',
 "Ma'an News Agency",
 'Maariv',
 'Makor Rishon',
 'Nocamels',
 'Palestine Chronicle',
 'Palestine News Network',
 'Shehab News Agency',
 'Shmuel Rosner',
 'Sikha Mekomit',
 'The Marker',
 'Times of Israel',
 'Wafa (Palestinian News and Info Agency) - Arabic',
 'Wafa (Palestinian News and Info Agency) - English',
 'Ynet - English',
 'Ynet - Hebrew',
 'Yediot',
 'Zman Israel',
 'Mida']

load `output.json` as a data frame

In [36]:
df = pd.read_json(JSON_WITH_IDS)
df= df.transpose()
df.set_index('id', inplace=True)

Find the rows that belong to the in scope publishers

In [37]:
in_scope_filt = df['name/title'].isin(scope)

In [38]:
in_scope_df = df[in_scope_filt]

Each in scope publisher has the ids of the articles that cite it `referrig record id`

For each in scope publisher row, create a row with `id` `Citation url or text alias` `Citation name/title`

example

lets say our in scope publishers contain `cnn.com`
lets say `almonitor.com/news` cites `cnn.com/weather`
the id for `almonitor.com/news` is 123
then we have

| id | Citation url or text alias | Citation name/title |
| --- | --- | --- |
| 123 | cnn.com/weather| cnn.com |

In [39]:
citation_obj ={}
citation_obj['id']=[]
citation_obj['Citation url or text alias'] =[]
citation_obj['Citation name/title'] =[]

{

    'column1':[row1col1,row2col1 ...],
    
    'column2':[row1col2,row2col2 ...],
    
    'column3':[row1col3,row2col3 ...]
}


In [40]:
def create_citiation_row(row):
    ids = row['referring record id']
    for ref_id in ids:
        citation_obj['id'].append(ref_id)
        citation_obj['Citation url or text alias'].append(row['url or alias text'])
        citation_obj['Citation name/title'].append(row['name/title'])
        

In [41]:
in_scope_df.apply(create_citiation_row, axis=1)

id
a57baf2b-0db5-5fda-a24a-fcab12ec1769    None
e5863206-648a-593e-b91c-cabc663aaab0    None
61885401-458c-56f4-98de-f522b349986d    None
93e5f857-2968-5753-8b40-32d95adc52b0    None
9e1b299e-6aa4-55ea-8d85-fe4207f7b94c    None
                                        ... 
65c906a9-67d2-5161-acd8-ac4455743a33    None
39676ac5-4db8-5ec7-9ce2-371726718288    None
11a496aa-8d87-5c2b-bc8c-66e54b6de7e3    None
8274ae94-1e8e-57a0-b37a-15b150b2f446    None
38a65c65-738a-5a01-a628-5f318a743392    None
Length: 30780, dtype: object

Convert the python dict to a dataframe

In [42]:
citation_df = pd.DataFrame(citation_obj)

In [43]:
citation_df

,id,Citation url or text alias,Citation name/title
0,1a4f6b47-c19b-5f89-bbc7-6f11d44c14a6_163766369...,https://www.haaretz.com/1.4717033,Haaretz - Hebrew
1,3a6f6126-1543-525f-a469-c4e79fe73e9e_163769889...,https://www.haaretz.com/1.4717033,Haaretz - Hebrew
2,1a4f6b47-c19b-5f89-bbc7-6f11d44c14a6_163766369...,https://www.haaretz.com/opinion/editorial/the-...,Haaretz - Hebrew
3,3a6f6126-1543-525f-a469-c4e79fe73e9e_163769889...,https://www.haaretz.com/opinion/editorial/the-...,Haaretz - Hebrew
4,1a4f6b47-c19b-5f89-bbc7-6f11d44c14a6_163766369...,https://www.timesofisrael.com/final-text-of-je...,Times of Israel
...,...,...,...
59232,471ddd79-1a6d-5ec1-a3d9-5c6ad389a749_163765655...,['Mida'],Mida
59233,dda29d96-3f47-53ae-8365-710755fcce1e_163794278...,['Mida'],Mida
59234,1445be27-d342-5d66-bec2-ff3401b24f5b_163769171...,['Mida'],Mida
59235,3f74fdb8-afd9-52b5-bc03-034a7c410136_163774852...,['Mida'],Mida


# Aggregate results to one row
- if you dont want the output in this format you can remove all the code blocks untill the Seperate Rows section

ids will repeat since each site could have cited multiple sources

so first group rows with the same ids together

In [44]:
cg = citation_df.groupby(['id'])

In [45]:
def test_agg(row):
    return row.tolist()

this merges the group rows in to a list. The `name/title` and the corresponding `url or text alias` have the same index in their corresponding lists. 

example
so if ` ynetnews.com/weather` is at index 0 in `url or text alias` then its corresponding `name/title` will also be at index 0

`[ynetnews.com/weather, ... ,cnn.com/news]`

`[Ynet-english, ..., cnn]`

In [46]:
cg =cg.agg(test_agg)

In [47]:
cg

,Citation url or text alias,Citation name/title
id,,
00019674-4c75-5e34-b986-1ce7be52b203_1637685489472,"[http://www.ynetnews.com/articles/0,7340,L-487...","[Ynet - English, Haaretz - Hebrew]"
00087cb1-7ace-5198-89f7-18718b58a9fb_1637714464291,"[http://www.israelhayom.co.il/article/380549, ...","[Israel Hayom - Hebrew, Israel Hayom - English]"
000a3d12-ef18-5a77-af56-1a4728e64149_1637651049516,"[https://www.inn.co.il/News/News.aspx/363030, ...","[Arutz 7 - Hebrew, Israel Hayom - Hebrew, Maar..."
0013dc9f-3521-5d2a-b0dd-816b719d2616_1637753174805,[http://www.timesofisrael.com/idf-bat-ayin-res...,"[Times of Israel, Times of Israel, Jerusalem P..."
001b831e-050f-5abd-9387-499f563c74ba_1637616878941,[https://www.haaretz.co.il/news/politics/.prem...,[Haaretz - Hebrew]
...,...,...
fff73490-7acc-5684-8671-68f6d5c0b202_1637941524916,[http://www.maannews.net/Content.aspx?id=897809],[Ma'an News Agency]
fff77c24-8761-5a9b-a13e-9420d0cb44b4_1638056548822,[http://www.jpost.com/landedpages/printarticle...,"[Jerusalem Post, Jerusalem Post, Arutz 7 - Eng..."
fffd690b-855d-5519-92d0-100636ab7f9f_1637887377946,[http://www.haaretz.co.il/literature/poetry/.p...,"[Haaretz - Hebrew, Maariv, Haaretz - Hebrew]"


next we append these columns to the original data frame

In [48]:
result = pd.concat([df,cg], axis='columns')

Then we get rid of any rows where the `Citation name/title` is na. These rows do not cite the sources in our scope

In [49]:
filt_nan_citation = ~result['Citation name/title'].isna()

In [50]:
result[filt_nan_citation][['referring record id','url or alias text','Citation url or text alias','Citation name/title']]

,referring record id,url or alias text,Citation url or text alias,Citation name/title
a6894d63-4229-516c-98fd-9118b186db39_1637772841749,[b92089ac-1305-58a2-843e-ee6ca8a58d95_16377315...,https://www.al-monitor.com/pulse/originals/202...,[https://www.timesofisrael.com/court-rules-all...,"[Times of Israel, Ynet - English, Times of Isr..."
b539cdfe-51c4-5adb-ab30-095e0f9904f6_1637657855324,[bec162ed-b70a-5699-bac4-c0dcbfd404d0_16376231...,https://www.al-monitor.com/originals/2019/10/i...,[https://www.israelhayom.com/2019/09/19/pm-can...,"[Israel Hayom - Hebrew, Arutz 7 - English, Tim..."
989935fe-ff58-5257-8c6f-40abc05b2f00_1637643663241,[16b9d86a-a3cd-517e-aa08-257f3893d004_16376106...,https://www.al-monitor.com/iw/contents/article...,"[https://www.ynet.co.il/articles/0,7340,L-5480...","[Ynet - Hebrew, Ynet - Hebrew, Maariv, Maariv,..."
054c7bb3-6c7a-5907-8a9f-3350027bc390_1637875112565,[37371f3f-09d7-5c4e-8af2-257751327e2e_16381411...,https://www.al-monitor.com/originals/2016/12/i...,"[http://www.ynetnews.com/articles/0,7340,L-470...",[Ynet - English]
e42efeeb-6e64-5434-82e3-a372296b384d_1637753863714,[5ef81516-229d-5a6b-97bd-dac7bb4d96af_16376976...,https://www.al-monitor.com/pulse/originals/201...,"[http://info.wafa.ps/atemplate.aspx?id=3458, h...",[Wafa (Palestinian News and Info Agency) - Eng...
...,...,...,...,...
302017e6-5617-5431-ad67-b9239da5aaac_1637877485501,[c618a31f-24ac-56a8-8361-b8c2bcdab95f_16380479...,https://www.al-monitor.com/originals/2014/04/i...,"[http://www.ynetnews.com/articles/0,7340,L-447...","[Ynet - English, Jerusalem Post, Jerusalem Post]"
70652b7e-8971-57bb-9f5d-a40952f8ba9a_1637641792565,[6ff8c71b-82a3-5198-a4b6-c0f15048484b_16376418...,https://www.al-monitor.com/pulse/originals/201...,"[https://www.ynetnews.com/articles/0,7340,L-50...","[Ynet - English, Times of Israel]"
9063a02e-483f-59b4-8432-65efb3d00e62_1637960508147,[5a597bfc-5dfa-5243-8f0f-d3e9d8cf602e_16379522...,https://www.al-monitor.com/iw/contents/article...,"[http://www.ynet.co.il/articles/0,7340,L-48260...","[Ynet - Hebrew, Haaretz - Hebrew]"
4f704222-87ad-5505-be5f-e324da9450c4_1638060651441,[11b993d3-dd4b-5aae-b3aa-1cd9865bf026_16380607...,https://www.al-monitor.com/originals/2013/05/n...,[http://www.haaretz.com/news/national/netanyah...,"[Haaretz - Hebrew, Ynet - English, Haaretz - E..."


# Results on seperate rows instead of aggregating

reset the index so that `id` isn't an index

In [51]:
df = df.reset_index()

join citation_df and df on id. This is equivalent to an SQL Inner JOIN on `id`

In [52]:
merged = citation_df.merge(df, on='id', how='inner', suffixes=('_url_citation_df','_df'))

move `Citation url or text alias` and `Citation name/title` to the back

In [53]:
merged['Citation url or text alias'], merged['Citation name/title'] = merged.pop('Citation url or text alias'), merged.pop('Citation name/title')

# Make any data manipulations here
- make any modifications to `merged` here
- ensure the final variable name is `merged_final`
- see `/juputer_notebooks/utils` for examples

finally sort by `url or alias text'`

In [54]:
merged_final = merged.sort_values(by=['url or alias text'])

In [55]:
merged_final

,id,url or alias text,referring record id,number of referrals,type,associated publisher,tags,name/title,language,authors,date of publication,plain text,image reference,anchor text,Citation url or text alias,Citation name/title
42603,9c962cc3-5789-58b1-bb48-759c9e235bf6_163766583...,https://www.al-monitor.com/afp/2017/02/israel-...,[2425ef51-6da9-5af7-991a-0526c0f44c37_16376316...,1,article,Al-Monitor,"[International, Referring, , United States, NA...",Al-Monitor,,Ben Caspit\n \n\n \n @BenCaspit,,Pfizer CEO Albert Bourla couldn't have dreamed...,,,https://www.timesofisrael.com/negative-tests-w...,Times of Israel
42602,9c962cc3-5789-58b1-bb48-759c9e235bf6_163766583...,https://www.al-monitor.com/afp/2017/02/israel-...,[2425ef51-6da9-5af7-991a-0526c0f44c37_16376316...,1,article,Al-Monitor,"[International, Referring, , United States, NA...",Al-Monitor,,Ben Caspit\n \n\n \n @BenCaspit,,Pfizer CEO Albert Bourla couldn't have dreamed...,,,http://www.timesofisrael.com/netanyahu-on-covi...,Times of Israel
42601,9c962cc3-5789-58b1-bb48-759c9e235bf6_163766583...,https://www.al-monitor.com/afp/2017/02/israel-...,[2425ef51-6da9-5af7-991a-0526c0f44c37_16376316...,1,article,Al-Monitor,"[International, Referring, , United States, NA...",Al-Monitor,,Ben Caspit\n \n\n \n @BenCaspit,,Pfizer CEO Albert Bourla couldn't have dreamed...,,,http://www.timesofisrael.com/pfizer-ceo-postpo...,Times of Israel
42600,9c962cc3-5789-58b1-bb48-759c9e235bf6_163766583...,https://www.al-monitor.com/afp/2017/02/israel-...,[2425ef51-6da9-5af7-991a-0526c0f44c37_16376316...,1,article,Al-Monitor,"[International, Referring, , United States, NA...",Al-Monitor,,Ben Caspit\n \n\n \n @BenCaspit,,Pfizer CEO Albert Bourla couldn't have dreamed...,,,https://www.timesofisrael.com/poll-signals-sli...,Times of Israel
13580,95b56142-e940-55c7-abde-10e5a263091e_163769689...,https://www.al-monitor.com/afp/2017/04/israel-...,[878c1817-9820-51b2-9c85-ab5b258338d3_16376616...,1,article,Al-Monitor,"[International, Referring, , United States, NA...",Al-Monitor,,Shlomi Eldar\n \n\n \n @@shlomieldar,,Mohammed Dahlan is known for his expertise in ...,,,https://www.jpost.com/Israel-News/Report-Mike-...,Jerusalem Post
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59166,e2713c54-dfbd-51ab-9f27-06d3988f0b65_163813822...,https://www.al-monitor.com/usa?_wrapper_format...,[69eb8ee6-a5ac-5072-b97d-3aaa6114bc45_16381382...,1,article,Al-Monitor,"[International, Referring, , United States, NA...",Al-Monitor,,Marian Houk\n \n\n \n @@Marianhouk,2021-12-02T12:00:00.000Z,"It was a significant step, as Palestinian Pres...",,,"['Wafa', 'Palestinian News and Info Agency']",Wafa (Palestinian News and Info Agency) - English
59191,205e15e4-82f2-54c2-8baf-ed97d1a45b23_163796633...,https://www.al-monitor.com/usa?_wrapper_format...,[dfd0b5cc-9465-533b-85ae-5d71d98c159d_16379656...,1,article,Al-Monitor,"[International, Referring, , United States, NA...",Al-Monitor,,"Jared Szuba | United Arab Emirates | Nov 20, 2021",2021-12-02T12:00:00.000Z,\n \n \n \n \n \n \n\n ...,,,"['Ynet', 'Yediot', 'Yedioth', 'ynetnews']",Ynet - English
58769,dd206b12-f867-5158-aa77-b4697c220c76_163805034...,https://www.al-monitor.com/usa?_wrapper_format...,[cf654430-b1fe-528f-8135-1e82abba41f6_16380500...,1,article,Al-Monitor,"[International, Referring, , United States, NA...",Al-Monitor,,-,2021-12-02T12:00:00.000Z,"אולי אין זה אלא מקרה, אך קו שחור עבה עובר בין ...",,,['הארץ'],Haaretz - Hebrew
58857,5550f771-f992-5ca7-b558-c50b4986a6bb_163805083...,https://www.al-monitor.com/usa?_wrapper_format...,[f8dadac1-e411-5b30-af85-de3b4561ff0d_16380505...,1,article,Al-Monitor,"[International, Referring, , United States, NA...",Al-Monitor,,"Jared Szuba | United Arab Emirates | Nov 20, 2021",2021-12-02T12:00:00.000Z,\n \n \n \n \n \n \n\n ...,,,"['Israel Hayom', 'IsraelHayom']",Israel Hayom - English


# Output
- output is found in `/juputer_notebooks/csv_processing/csv_output/`
- panadas can export to different formats (csv, json excel)
- remove old output file if they are no longer needed

Export to excel

In [56]:
date_string = datetime.now().strftime("%Y_%m_%d")

In [57]:
OUTPUT_SEPERATE_ROWS_PATH = "./csv_output/almontior_seperate_rows_"

In [ ]:
OUTPUT_AGG_PATH = './csv_output/almonitor_agg_one_row_'

In [ ]:
merged_final.to_excel(OUTPUT_SEPERATE_ROWS_PATH+date_string+'.xlsx')

In [ ]:
result[filt_nan_citation].to_excel(OUTPUT_AGG_PATH+date_string+'.xlsx')